In [5]:
# https://nb.recohut.com/jupyter/pytorch/2021/04/21/rec-algo-ncf-pytorch-pyy0715.html
# https://colab.research.google.com/github/recohut/notebook/blob/master/_notebooks/2021-04-21-rec-algo-ncf-pytorch-pyy0715.ipynb#scrollTo=VT89-ZSZ9pMl
import os
import time
import random
import argparse
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tensorboardX import SummaryWriter

In [12]:
from utils import hit, ndcg, metrics, seed_everything
from model import Generalized_Matrix_Factorization, Multi_Layer_Perceptron, NeuMF
from CF_data import NCF_Data
from data_loader import Rating_Datset
from constants import *

ImportError: cannot import name 'seed_everything' from 'utils' (C:\Users\trlon\PycharmProjects\pythonProject\recommendation\pytorch\utils.py)

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--seed", 
    type=int, 
    default=42, 
    help="Seed")
parser.add_argument("--lr", 
	type=float, 
	default=0.001, 
	help="learning rate")
parser.add_argument("--dropout", 
	type=float,
	default=0.2,  
	help="dropout rate")
parser.add_argument("--batch_size", 
	type=int, 
	default=256, 
	help="batch size for training")
parser.add_argument("--epochs", 
	type=int,
	default=10,  
	help="training epoches")
parser.add_argument("--top_k", 
	type=int, 
	default=10, 
	help="compute metrics@top_k")
parser.add_argument("--factor_num", 
	type=int,
	default=32, 
	help="predictive factors numbers in the model")
parser.add_argument("--layers",
    nargs='+', 
    default=[64,32,16,8],
    help="MLP layers. Note that the first layer is the concatenation of user \
    and item embeddings. So layers[0]/2 is the embedding size.")
parser.add_argument("--num_ng", 
	type=int,
	default=4, 
	help="Number of negative samples for training set")
parser.add_argument("--num_ng_test", 
	type=int,
	default=100, 
	help="Number of negative samples for test set")
parser.add_argument("--out", 
	default=True,
	help="save model or not")


_StoreAction(option_strings=['--out'], dest='out', nargs=None, const=None, default=True, type=None, choices=None, help='save model or not', metavar=None)

In [11]:
args = parser.parse_args("")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter()

# seed for Reproducibility
seed_everything(args.seed)

# load data
ml_1m = pd.read_csv(
    DATA_PATH, 
    sep="::", 
    names = ['user_id', 'item_id', 'rating', 'timestamp'], 
    engine='python')

# set the num_users, items
num_users = ml_1m['user_id'].nunique()+1
num_items = ml_1m['item_id'].nunique()+1

# construct the train and test datasets
data = NCF_Data(args, ml_1m)
train_loader = data.get_train_instance()
test_loader = data.get_test_instance()

# set model and loss, optimizer
model = NeuMF(args, num_users, num_items)
model = model.to(device)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# train, evaluation
best_hr = 0
for epoch in range(1, args.epochs+1):
    model.train() # Enable dropout (if have).
    start_time = time.time()

    for user, item, label in train_loader:
        user = user.to(device)
        item = item.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        prediction = model(user, item)
        loss = loss_function(prediction, label)
        loss.backward()
        optimizer.step()
        writer.add_scalar('loss/Train_loss', loss.item(), epoch)

    model.eval()
    HR, NDCG = metrics(model, test_loader, args.top_k, device)
    writer.add_scalar('Perfomance/HR@10', HR, epoch)
    writer.add_scalar('Perfomance/NDCG@10', NDCG, epoch)

    elapsed_time = time.time() - start_time
    print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
            time.strftime("%H: %M: %S", time.gmtime(elapsed_time)))
    print("HR: {:.3f}\tNDCG: {:.3f}".format(np.mean(HR), np.mean(NDCG)))

    if HR > best_hr:
        best_hr, best_ndcg, best_epoch = HR, NDCG, epoch
        if args.out:
            if not os.path.exists(MODEL_PATH):
                os.mkdir(MODEL_PATH)
            torch.save(model, 
                '{}{}.pth'.format(MODEL_PATH, MODEL))

writer.close()

NameError: name 'seed_everything' is not defined